In [18]:

import os
import pandas as pd

def normalize_name(filename):
    return filename.replace("olist_", "").replace("_dataset", "").replace(".csv", "")

def load_all_data(folder_path):
    try:
        files = [f for f in os.listdir(folder_path) if f.endswith(".csv")]
        data = {normalize_name(filename): pd.read_csv(os.path.join(folder_path, filename)) for filename in files}
        return data
    except Exception as e:
        print(f"Error al cargar los datos: {str(e)}")
        return None


In [19]:
import pandas as pd

import pandas as pd

import pandas as pd

def transformar_columnas_datetime(df):
    columnas_de_fechas = ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date',
                          'order_delivered_customer_date', 'order_estimated_delivery_date']

    try:
        for columna in columnas_de_fechas:
            df[columna] = pd.to_datetime(df[columna])
        print(f"Columnas {columnas_de_fechas} convertidas a formato de datetime.")
    except Exception as e:
        print(f"Error al convertir las columnas a datetime: {str(e)}")
    return df




In [20]:
import pandas as pd

def calcular_tiempo_de_espera():

    try:
        orders['tiempo_de_espera'] = orders['order_delivered_customer_date'] - orders['order_purchase_timestamp']
        orders['tiempo_de_espera'] = orders['tiempo_de_espera'].dt.days


        # Verificación
        print(orders[['order_purchase_timestamp', 'order_delivered_customer_date', 'tiempo_de_espera']])
        print("Columna 'tiempo_de_espera' creada y diferencia de tiempo convertida a días.")
    except Exception as e:
        print(f"Error al calcular el tiempo de espera: {str(e)}")




In [21]:
#import pandas as pd

#def calcular_diferencia_real_vs_esperado(df, columna_real, columna_esperado):
 #   try:

  #      diferencia = (df[columna_real] - timedelta(days=df[columna_esperado]))
        #.dt.days

        #negativos a 0
        #diferencia = diferencia.clip(lower=0)


   #     df['real_vs_esperado'] = diferencia

    #except Exception as e:
     #   print(f"Error al calcular la diferencia entre fechas: {str(e)}")






In [22]:
import pandas as pd

def calcular_real_vs_esperado(df):
    try:
        # Calcula la diferencia en días entre order_delivered_customer_date y fecha_esperada
        diferencia = df["tiempo_de_espera"] - df["tiempo_de_espera_previsto"]

        # Convierte valores negativos a 0 y almacénalos en la columna 'real_vs_esperado'
        df['real_vs_esperado'] = diferencia.clip(lower=0)

    except Exception as e:
        print(f"Error al calcular la diferencia entre fechas: {str(e)}")



In [23]:
import numpy as np

def tiempo_de_espera(orders, is_delivered=True):
    # filtrar por entregados y crea la variable tiempo de espera
    if is_delivered:
        orders = orders.query("order_status=='delivered'").copy()
    # calcular tiempo de espera
    orders.loc[:, 'tiempo_de_espera'] = \
        (orders['order_delivered_customer_date'] -
         orders['order_purchase_timestamp']) / np.timedelta64(24, 'h')
    return orders

In [24]:
import numpy as np

def tiempo_de_espera_previsto(orders, is_delivered=True):
    # filtrar por entregados y crea la variable tiempo de espera
    if is_delivered:
        orders = orders.query("order_status=='delivered'").copy()
    # calcular tiempo de espera
    orders.loc[:, 'tiempo_de_espera_previsto'] = \
        (orders['order_estimated_delivery_date'] -
         orders['order_purchase_timestamp']) / np.timedelta64(24, 'h')
    return orders

In [25]:
def puntaje_de_compra(reviews):
    reviews['orden_original'] = range(len(reviews))

    def asignar_una_estrella(x):
        return x.eq(1).any().astype(int)

    def asignar_cinco_estrellas(x):
        return x.eq(5).all().astype(int)

    resultados = pd.DataFrame({
        'order_id': reviews['order_id'],
        'es_cinco_estrellas': reviews.groupby('order_id')['review_score'].transform(asignar_cinco_estrellas),
        'es_una_estrella': reviews.groupby('order_id')['review_score'].transform(asignar_una_estrella),
        'review_score_original': reviews['review_score'],

    })


    return resultados





In [26]:
def generar_data(df1, df2, columna):
    data = df1.merge(df2, on=columna, how="left")
    return data


In [27]:
def calcular_numero_productos(data):
  merge_sorted = data.groupby("order_id").agg(number_of_products = ("product_id","count")).reset_index()
  return merge_sorted


In [28]:
def vendedores_unicos(data):
  return data.groupby("order_id").agg(vendedores_unicos = ("seller_id","nunique")).reset_index()


In [29]:
def calcular_precio_y_transporte(data):

    data = data.groupby("order_id").agg({
        'price': 'sum',
        'freight_value': 'sum'
    }).reset_index()


    data.rename(columns={'price': 'precio', 'freight_value': 'transporte'}, inplace=True)

    return data


In [30]:
from math import radians, sin, cos, asin, sqrt

def haversine_distance(lon1, lat1, lon2, lat2):
    """
    Computa distancia entre dos pares (lat, lng)
    Ver - (https://en.wikipedia.org/wiki/Haversine_formula)
    """
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    return 2 * 6371 * asin(sqrt(a))


In [31]:
 def distancia_al_cliente(data):
  data['distance_seller_customer'] = data.apply(lambda row: haversine_distance(row['geolocation_lng'], row['geolocation_lat'],  row['custo_geolocation_lng_y'], row['custo_geolocation_lat_y']), axis=1)
  data = data.dropna()
  return data.groupby("order_id")["distance_seller_customer"].mean().reset_index()